In [ ]:
import pandas as pd
import os
from queryhandler.database_connector import DatabaseConnector

In [ ]:
# connect to database in order to fetch top 200 coins data
db_connector = DatabaseConnector(
    db_name=os.environ.get("DATABASE_NAME"),
    db_user=os.environ.get("DATABASE_USER"),
    db_password=os.environ.get("DATABASE_PASSWORD"),
    db_host=os.environ.get("DATABASE_HOST"),
    db_port=os.environ.get("DATABASE_PORT"),
)
engine, connection = db_connector.connect_to_db()


In [26]:
# define the query to fetch data from coin table
query = "SELECT * FROM coin_history where time_open='2023-08-25'"
df = pd.read_sql(query, engine)

# plot figure
ax = df.plot.scatter(x="market_cap", y="volume")
ax.set_title("Market-cap and Volume Relation")

,id,coin,date,market_cap,open,rank,low,high,close,volume,time_open,time_close,time_high,time_low
0,13,28,None,2510937181,7.0,None,7.0,7.0,7.0,8.742949e+07,2023-08-25,2023-08-25 23:59:59,2023-08-25 12:13:00,2023-08-25 04:53:00
1,377,172,None,116876111,0.0,None,0.0,0.0,0.0,1.078057e+06,2023-08-25,2023-08-25 23:59:59,2023-08-25 03:03:00,2023-08-25 07:59:00
2,741,7,None,9152956074,0.0,None,0.0,0.0,0.0,2.267454e+08,2023-08-25,2023-08-25 23:59:59,2023-08-25 02:05:00,2023-08-25 20:08:00
3,1105,78,None,408533686,25.0,None,24.0,25.0,25.0,4.328138e+07,2023-08-25,2023-08-25 23:59:59,2023-08-25 19:19:00,2023-08-25 05:05:00
4,1469,164,None,134943892,1.0,None,1.0,1.0,1.0,1.534114e+07,2023-08-25,2023-08-25 23:59:59,2023-08-25 23:57:00,2023-08-25 04:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,67760,54,None,635749025,1.0,None,1.0,1.0,1.0,9.643727e+06,2023-08-25,2023-08-25 23:59:59,2023-08-25 22:10:00,2023-08-25 04:43:00
193,68124,36,None,1293223854,6.0,None,6.0,6.0,6.0,8.839688e+07,2023-08-25,2023-08-25 23:59:59,2023-08-25 00:17:00,2023-08-25 15:04:00
194,68447,57,None,582258459,0.0,None,0.0,0.0,0.0,2.998064e+07,2023-08-25,2023-08-25 23:59:59,2023-08-25 00:00:00,2023-08-25 15:04:00
195,68811,162,None,136032462,0.0,None,0.0,0.0,0.0,6.617848e+06,2023-08-25,2023-08-25 23:59:59,2023-08-25 19:16:00,2023-08-25 05:05:00
